In [ ]:
from autoemulate.emulators import GaussianProcess
from autoemulate.core.compare import AutoEmulate
from autoemulate.simulations.epidemic import Epidemic
from autoemulate.calibration.history_matching import HistoryMatchingWorkflow
from autoemulate.calibration.history_matching_dashboard import HistoryMatchingDashboard


In [ ]:
random_seed = 42

# History Matching workflow

The `HistoryMatching` calibration method can be a useful way to iteratively decide which simulations to run to generate data refine the emulator on. The `HistoryMatchingWorkflow` implements this iterative sample-predict-refit workflow. Each time it is run:
- parameters are sampled from the not ruled out yet (NROY) space
- an emulator is used in combination with `HistoryMatching` to score the implausability of the samples
- simulations are run for a subset of the NROY samples
- the emulator is refit given the newly simulated data


In this tutorial, we demonstrate how to implement this simulator in the loop workflow.


## 1. Simulate data and train an emulator

In this example, we'll use the `Epidemic` simulator, which returns the peak infection rate given two input parameters, `beta`(the transimission rate per day) and `gamma` (the recovery rate per day).

In [ ]:
simulator = Epidemic(log_level="error")
x = simulator.sample_inputs(500, random_seed=random_seed)
y = simulator.forward_batch(x)

For the purposes of this tutorial, we will restrict the model choice to `GaussianProcess`.

In [ ]:
ae = AutoEmulate(
    x,
    y,
    models=[GaussianProcess],
    model_tuning=False,
    log_level="error",
    random_seed=random_seed
)

We can verify that the fitted emulator performs well on both the train and test data.

In [ ]:
ae.summarise()

In [ ]:
best_result = ae.best_result()

## 2. Calibrate

To instantiate the `HistoryMatchingWorkflow` object, we need an observed mean and, optionally, variance for each simulator output.

In [ ]:
observations = {"infection_rate": (0.3, 0.05)}

We also pass the fitted emulator and the simulator to the `HistoryMatchingWorkflow` along with the observations. 

The user can also set the usually history matching parameters (e.g., implausability threshold). Optionally, the user can also pass the training data used to fit the emulator, which can then be used in refitting the emulator in combination with newly simulated data during the history matching workflow.

In [ ]:
hmw = HistoryMatchingWorkflow(
    simulator=simulator,
    result=best_result,
    observations=observations,
    threshold=3.0,
    random_seed=random_seed
)


The `run` method implements the iterative sample-predict-refit workflow:
- sample `n_test_samples` to test from the not ruled out yet (NROY) space
- use emulator to filter out implausible samples and update the NROY space
- run `n_simulations` predictions for the sampled parameters using the simulator

Optionally, the `run` method can also `refit_the_emulator` using only the newly simulated data or all the data available so far (see `refit_on_all_data` parameter).

In [ ]:
test_parameters, impl_scores = hmw.run(n_simulations=20, n_test_samples=100)

The `HistoryMatchingWorkflow` object maintains and updates the internal state each time `run()` is called, including saving the newly simulated data. This means the full workflow can be run over a number of iterations by calling `run` repeatedly.

Alternatively, you can use the `run_waves` method to run multiple waves of the history matching workflow in one go.

## 3. Visualise results

For visualising results, you can use the `HistoryMatchingDashboard` which implements a number of interactive plots. To initialize it just pass in outputs of the `run()` method and information about the simulation.

In [ ]:
dashboard = HistoryMatchingDashboard(
    samples=test_parameters,
    impl_scores=impl_scores,
    param_names=simulator.param_names,  
    output_names=simulator.output_names, 
)

The plots can be viewed using the `display()` method. Below you can see an example image of what the dashboard looks like.

In [ ]:
# dashboard.display()

<img src="https://raw.githubusercontent.com/alan-turing-institute/autoemulate/refs/heads/main/misc/vis_dashboard_pic_sample.png" alt="Work Flow" style="width:50%;"/> 